In [3]:
import pandas as pd
import numpy as np

seds = pd.read_csv('datasets/seds.csv') #1
revenue_to_provider = pd.read_csv('datasets/revenue_to_provider.csv') #2
filter_pack_concentrations_weekly = pd.read_csv('datasets/filter_pack_concentration_weekly.csv') #3
energy_workforce_training = pd.read_csv('datasets/energy_workforce_training.csv') #4
vehicles = pd.read_csv('datasets/vehicles.csv') #5
land_temp_by_city = pd.read_csv('datasets/land_temp_by_city.csv') #6
land_temp_by_state = pd.read_csv('datasets/land_temp_by_state.csv') #7
power_plants = pd.read_csv('datasets/power_plants.csv') #8

In [15]:
power_plants.head()

longitude   latitude          plant_name      utility_name  \
0 -87.356823  33.458665        Bankhead Dam  Alabama Power Co   
1 -88.010300  31.006900               Barry  Alabama Power Co   
2 -86.283056  32.583889  Walter Bouldin Dam  Alabama Power Co   
3 -85.970800  34.012800             Gadsden  Alabama Power Co   
4 -87.196486  33.644344              Gorgas  Alabama Power Co   

        sector_name       city  county state_name primary_source  total_power  \
0  Electric Utility  Northport      as    Alabama  hydroelectric         56.0   
1  Electric Utility      Bucks  Mobile    Alabama           coal       2160.5   
2  Electric Utility   Wetumpka  Elmore    Alabama  hydroelectric        225.9   
3  Electric Utility    Gadsden  Etowah    Alabama    natural gas        130.0   
4  Electric Utility    Parrish  Walker    Alabama           coal       1034.0   

                        source_description  \
0                                        g   
1  Coal = 1088.5 MW, Natural Gas = 1072 MW   
2                 Hydroelectric = 225.9 MW   
3                     Natural Gas = 130 MW   
4                           Coal = 1034 MW   

                                    tech_description  
0                         Conventional Hydroelectric  
1  Conventional Steam Coal; Natural Gas Fired Com...  
2                         Conventional Hydroelectric  
3                          Natural Gas Steam Turbine  
4                            Conventional Steam Coal

In [16]:
power_plants.describe()

longitude     latitude  total_power
count  7938.000000  7938.000000  7826.000000
mean    -95.245314    39.134308   135.639215
std      18.669527     5.741334   357.091293
min    -171.712439   -17.287022     0.100000
25%    -112.859600    35.218360     3.025000
50%     -91.811438    39.545654    11.500000
75%     -79.305587    42.554413    81.000000
max     -38.914783    71.292000  7079.000000

In [10]:
primary_sources = set(power_plants['primary_source'])
print(primary_sources)

{nan, 'nuclear', 'other', 'wind', 'hydroelectric', 'natural gas', 'coal', 'pumped storage', 'biomass', 'solar', 'geothermal', 'petroleum'}


In [17]:
# PLANT TO GAS NEURAL NETWORK MODEL HERE:
from torch.autograd import Variable
from tensorboardX import SummaryWriter
import shutil
from tqdm import tqdm
import numpy as np
import utils

import os
import pdb
import pickle
import argparse

import warnings
warnings.filterwarnings("ignore")

# Torch imports
import torch
import torch.nn as nn
import torch.optim as optim

# Numpy & Scipy imports
import numpy as np
import scipy
import scipy.misc


def adjust_learning_rate(optimizer, epoch, params):
    """Sets the learning rate to the initial LR multiplied by 0.98 every epoch"""
    learning_rate = params['lr'] * (self.params['lr_decay'] ** (epoch//params['lr_step']))
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate
    return learning_rate

# Input plant types we care about (listed for easy tuning)
sources = ['nuclear', 'wind', 'hydroelectric', 'natural gas', 'coal', 'pumped storage', 'biomass', 'solar', 'geothermal']
# Feature gases that we care about
gases = ['TSO4','TNH4','CA','MG','NA','K','CL','NSO4','NHNO3','WSO2','WNO3']
gases.remove('MG')
gases.remove('NA')
gases.remove('K')
params = {'hidden_dim': 10, 'lr': 0.01, 'lr_decay': 0.99, 'lr_step': 1, 'num_iters': 1000}
model = nn.Sequential(nn.Linear(len(sources), hidden_dim),
                     nn.ReLU(),
                     nn.Linear(params['hidden_dim'], n_out))
loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters, lr=params['lr'])
for epoch in range(50):
    # Forward Propagation
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print('epoch: ', epoch,' loss: ', loss.item())

    # Zero the gradients
    optimizer.zero_grad()
    
    # perform a backward pass (backpropagation)
    loss.backward()
    
    # Update the parameters
    optimizer.step()




TypeError: remove() takes exactly one argument (2 given)

In [8]:
seds_energy_bins = set(seds['energy_bin'])
print(seds_energy_bins)

{'Petroleum', 'Natural Gas', 'Nuclear', 'Renewable', 'Coal', 'Electricity', 'Aggregate'}


In [12]:
revenue_sectors = set(revenue_to_provider['industry_sector'])
print(revenue_sectors)
power_plant_sectors = set(power_plants['sector_name'])
print(power_plant_sectors)

{'Energy-Only Providers', 'Restructured Retail Service Providers', 'Total Electric Industry', 'Delivery-Only Service', 'Full-Service Providers'}
{nan, 'nbn', 'Commercial CHP', 'IPP CHP', 'Commercial Non-CHP', 'Electric Utility', 'IPP Non-CHP', 'Industrial CHP', 'Industrial Non-CHP'}


In [13]:
training_certificates = set(energy_workforce_training['certificate'])
print(training_certificates)

{'Alternative Energy', 'Electronics,', 'Energy Systems for Precision Manufacturing', 'Precision Welding', 'Mechatronics ', 'Nanoscience Technician ', 'Electromechanical Technology', 'Microsystems', 'Robotic ARC Welder', 'Transportationportation and Material Moving', 'SMAW', 'Biofuel Plant Technician', 'Engineering Technology', 'Welding Technology', 'Stem In Manufacturing', 'CNC Machining', 'Electrical Utility Technology ', 'Aircraft Assembly', 'Entrepreneurship', 'Floor Hand', 'PMMI Mechatronics Certification', 'Wind', 'Engineering PhD', 'Mechanical Maintenance', 'Electronics Engineering', 'Alternative Fuels ', 'STEM career Pathways', 'PLC  ', 'Oil Technology', 'Nanotechnology ', 'Airframe Technical Certificate', 'Smart Meter Technicians', 'Supply Chain Systems', 'Certificate In Renewable Energy', 'Geothermal', 'Waste Stream Reuse', 'Solar Fabrication Technician', 'Machine Technology', 'Alternate Energy Technician', 'Robotic Welding ', 'Industrial Technology', 'Renewable Energy Technic

In [15]:
set(power_plants['city'])

{nan,
 'East Hartford',
 'Tok',
 'Gorham',
 'Kingston Twp',
 'Oklaunion',
 'Biron',
 'Farmer City',
 'Rincon',
 'Sandersville',
 'Temple',
 'Phoenix',
 'Creswell',
 'St Landry',
 'Ellenboro',
 'Cherryville',
 'New Richmond',
 'Wocester',
 'Maidsville',
 'Belen',
 'Benton',
 'Burnsville',
 'Richland',
 'Kincaid',
 'Wapato',
 'Covert',
 'Sagamore',
 'Town of Johnston',
 'Eunicel',
 'Jackson',
 'Rockland',
 'Westlake',
 'La Habra',
 'Seneca Falls',
 'FREEBURG',
 'Pottsville',
 'Jewett',
 'Cherry Hill',
 'Ronkonkoma',
 'Centreville',
 'Dexter',
 'Cope',
 'Hyampom',
 'Albany',
 'Newark',
 'Dadeville',
 'Barnard',
 'Wilber',
 'Malta',
 'Reno',
 'Sterling City',
 'Otay Mesa',
 'City of Morganton',
 'Merrimac',
 'Rocky',
 'Hayward',
 'Pacolet',
 'Gibson',
 'Browns Valley',
 'Fort Morgan',
 'Beaver Creek',
 'Watsonville',
 'Hadley',
 'Casco',
 'Southwick',
 'Fremont',
 'Ft. Gaines',
 'Culver',
 'Spalding',
 'Nicholson',
 'Northwoods',
 'Coleman Falls',
 'Juliette',
 'Fleish',
 'Gladwin',
 'Crai